# Setup

In [1]:
# ver4.9

############################################
##### decide which real machine to use #####
############################################
backend_name = 'ibm_kawasaki'

############################
##### import libraries #####
############################
%matplotlib inline
%load_ext autoreload
%autoreload 2


from qiskit import *
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, compiler, circuit, compiler
from qiskit.converters import circuit_to_dag, dag_to_circuit
from pytket.extensions.qiskit import IBMQBackend, AerBackend
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import math
import time
import sys
sys.path.append('../../../aqcel_dev/aqcel_dev')
import aqcel_dev as icepp
sys.path.append('../')
sys.path.append('../qiskit_legacy')
sys.path.append('../qiskit_code')

import QuantumPartonShower as qps
import QuantumPartonShower_ReM as qpsc
import QuantumPartonShower_ReM_2step_hardcode as qps_rhs
import plotting as pl
from PaperPlots import MakeObservables as mo

sim_b = AerBackend()



displaycolor = {
    'u1': ('#FA74A6', '#000000'),
    'u2': ('#FA74A6', '#000000'),
    'u3': ('#FA74A6', '#000000'),
    'id': ('#191970', '#ffffff'),
    'u': ('#BB8BFF', '#000000'),
    'p': ('#BB8BFF', '#000000'),
    'x': ('#05BAB6', '#000000'),
    'y': ('#05BAB6', '#000000'),
    'z': ('#05BAB6', '#000000'),
    'h': ('#6FA4FF', '#000000'),
    'cx': ('#ff00ff', '#000000'),
    'cry': ('#ff00ff', '#000000'),
    'ch': ('#ff00ff', '#000000'),
    'ccx': ('#ff00ff', '#000000'),
    'mcx': ('#ff00ff', '#000000'),
    'mcx_gray': ('#ff00ff', '#000000'),
    'cy': ('#ff00ff', '#000000'),
    'cz': ('#ff00ff', '#000000'),
    'swap': ('#6FA4FF', '#000000'),
    'cswap': ('#ff00ff', '#000000'),
    'ccswap': ('#ff00ff', '#000000'),
    'dcx': ('#ff00ff', '#000000'),
    'cdcx': ('#ff00ff', '#000000'),
    'ccdcx': ('#ff00ff', '#000000'),
    'iswap': ('#6FA4FF', '#000000'),
    's': ('#6FA4FF', '#000000'),
    'sdg': ('#6FA4FF', '#000000'),
    't': ('#BB8BFF', '#000000'),
    'tdg': ('#BB8BFF', '#000000'),
    'sx': ('#6FA4FF', '#000000'),
    'sxdg': ('#6FA4FF', '#000000'),
    'r': ('#BB8BFF', '#000000'),
    'rx': ('#BB8BFF', '#000000'),
    'ry': ('#BB8BFF', '#000000'),
    'rz': ('#BB8BFF', '#000000'),
    'rxx': ('#BB8BFF', '#000000'),
    'ryy': ('#BB8BFF', '#000000'),
    'rzx': ('#BB8BFF', '#000000'),
    'reset': ('#000000', '#FFFFFF'),
    'target': ('#FFFFFF', '#FFFFFF'),
    'measure': ('#000000', '#FFFFFF'),
    'ccx_to_x': ('#006400', '#ffffff'),      # 緑
    'ccx_to_cx': ('#800000', '#ffffff'),     # 茶
    'ccx_to_removed': ('#006400', '#ffffff'),# 緑
    'cx_to_x': ('#191970', '#ffffff'),       # 青
    'cx_to_removed': ('#191970', '#ffffff'), # 青
}

style = {'displaycolor' : displaycolor, 'showindex' : True}

Number of emissions: 1
log(theta_max): -5.180816459236603
Number of emissions: 1
log(theta_max): -1.7269388197455342
Number of emissions: 2
log(theta_max): -1.7269388197455342
Number of emissions: 1
log(theta_max): -3.4538776394910684


# ①Original Cirucuit

In [ ]:
N = 2
eps= 0.001
g_1= 2
g_2= 1
initialParticles= ['100'] # f1

shots= 10000

qps0= qps.QuantumPartonShower(N, 1) # for g12= 0
qps1= qps.QuantumPartonShower(N, 1) # for g12= 1

qc0, qubits0= qps0.createCircuit(eps, g_1, g_2, 0, initialParticles)
qc1, qubits1= qps1.createCircuit(eps, g_1, g_2, 1, initialParticles)

t1= time.time()
counts0= qps0.simulate('qasm', shots=shots)
t2= time.time()
counts1= qps1.simulate('qasm', shots=shots)
t3= time.time()

generated circuit on 11 qubits
generated circuit on 11 qubits


# ③AQCEL circuit optimization

In [ ]:
#######################
##### apply aqcel #####
#######################

# AQCELにかかる時間を測定開始
start_time = time.time()

# AQCELのapply
target_qc = qc0
aqcel_results_cc= icepp.pass_manager(target_qc, level=2, backend_tket=sim_b, shots=1024, measure_type='cc').auto_manager()
result_qc = aqcel_results_cc[0]

qc1, qc2, qc3, qc4, qc6 = aqcel_results_cc[9], aqcel_results_cc[10], aqcel_results_cc[11], aqcel_results_cc[12], aqcel_results_cc[1]

# AQCEL適用にかかった時間をprint
end_time = time.time()
execution_time = end_time - start_time
print(f"実行時間: {execution_time}秒")

In [ ]:
#############################
##### Before AQCEL(qc) #####
#############################
qc = target_qc
print('Before AQCEL(qc)')
print(f' - Number of gate   : {list(target_qc.count_ops().items())}')
print(f' - Number of qubits : {target_qc.num_qubits}')
print(f' - Depth of all     : {target_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')


#############################
##### After decompose_mcu(qc1) #####
#############################
print('qc1 : After decompose_mcu')
print(f' - Number of gate   : {list(qc1.count_ops().items())}')
print(f' - Number of qubits : {qc1.num_qubits}')
print(f' - Depth of all     : {qc1.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_adjacent(qc2) #####
#############################
print('qc2 : After remove_adjacent')
print(f' - Number of gate   : {list(qc2.count_ops().items())}')
print(f' - Number of qubits : {qc2.num_qubits}')
print(f' - Depth of all     : {qc2.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_controlled_operations(qc3) #####
#############################
print('qc3 : After remove_controlled_operations')
print(f' - Number of gate   : {list(qc3.count_ops().items())}')
print(f' - Number of qubits : {qc3.num_qubits}')
print(f' - Depth of all     : {qc3.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

#############################
##### After remove_adjacent(qc4) #####
#############################
print('qc4 : After remove_adjacent')
print(f' - Number of gate   : {list(qc4.count_ops().items())}')
print(f' - Number of qubits : {qc4.num_qubits}')
print(f' - Depth of all     : {qc4.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

############################
##### After AQCEL(qc5) #####
############################
qc5 = result_qc
print('After AQCEL(qc5) : After remove_qubits')
print(f' - Number of gate   : {list(result_qc.count_ops().items())}')
print(f' - Number of qubits : {result_qc.num_qubits}')
print(f' - Depth of all     :{result_qc.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

############################
##### After transpiled(qc6) #####
############################
print('qc6 : After transpiled')
print(f' - Number of gate   : {list(qc6.count_ops().items())}')
print(f' - Number of qubits : {qc6.num_qubits}')
print(f' - Depth of all     :{qc6.depth()}')# 1から始めてmeasurementは含めないで数えてくれる
print('')
print('')

# かかる時間を測定開始
start_time = time.time()

import qiskit.quantum_info as qi
statevector1 = icepp.simulator(target_qc).statevector_simulator(output='statevector')
statevector2 = icepp.simulator(result_qc).statevector_simulator(output='statevector')
dm1 = qi.partial_trace(statevector1,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm1.probabilities_dict()
dm2 = qi.partial_trace(statevector2,[0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 17, 18, 19])
dm2.probabilities_dict()

# かかった時間をprint
end_time = time.time()
execution_time = end_time - start_time
print(f"実行時間: {execution_time}秒")
print(f"Fsim = {qi.state_fidelity(dm1,dm2)}")

In [ ]:
#############################
##### Before AQCEL(qc) #####
#############################
qc = target_qc
print('Before AQCEL(qc)')
# qc.draw('mpl', initial_state = True, style = style, )


# target_qc.draw('mpl',fold = 100, initial_state = True, style = style)

In [ ]:
dag = circuit_to_dag(target_qc)
predecessors = []

for node in dag.topological_op_nodes():
    if node.name == 'ccx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-ccx'
            predecessors.append(predecessor)
            predecessor = list(dag.predecessors(node))[2].name + '-ccx'
            predecessors.append(predecessor)
        except:
            predecessor = 'Initial state-ccx'
            predecessors.append(predecessor)
    elif node.name == 'cx' or node.name == 'ch' or node.name == 'rcx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-cx'
        except:
            predecessor = 'Initial state-cx'
        predecessors.append(predecessor)
print(f'predecessors : {predecessors}')

# データの頻度を計算
counter = Counter(predecessors)
# データと頻度を分解
labels, values = zip(*counter.items())
# 棒グラフを描画
plt.bar(labels, values)
# 軸ラベルとタイトルを設定
plt.xlabel('predecessor gate')
plt.ylabel('Frequency')
plt.title('Histogram of predecessor gate')
# x軸のラベルを90度回転
plt.xticks(rotation=90)
# グラフを表示
plt.show()

In [ ]:
#############################
##### After decompose_mcu(qc1) #####
#############################
print('qc1 : After decompose_mcu')
# qc1.draw('mpl',fold = 100, initial_state = True, style = style)

In [ ]:
dag = circuit_to_dag(qc1)
predecessors = []

for node in dag.topological_op_nodes():
    if node.name == 'ccx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-ccx'
            predecessors.append(predecessor)
            predecessor = list(dag.predecessors(node))[2].name + '-ccx'
            predecessors.append(predecessor)
        except:
            predecessor = 'Initial state-ccx'
            predecessors.append(predecessor)
    elif node.name == 'cx' or node.name == 'ch' or node.name == 'rcx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-cx'
        except:
            predecessor = 'Initial state-cx'
        predecessors.append(predecessor)
print(f'predecessors : {predecessors}')

# データの頻度を計算
counter = Counter(predecessors)
# データと頻度を分解
labels, values = zip(*counter.items())
# 棒グラフを描画
plt.bar(labels, values)
# 軸ラベルとタイトルを設定
plt.xlabel('predecessor gate')
plt.ylabel('Frequency')
plt.title('Histogram of predecessor gate')
# x軸のラベルを90度回転
plt.xticks(rotation=90)
# グラフを表示
plt.show()

In [ ]:
#############################
##### After remove_adjacent(qc2) #####
#############################
print('qc2 : After remove_adjacent')
# qc2.draw('mpl',fold = 100, initial_state = True, style = style)

In [ ]:
dag = circuit_to_dag(qc2)
predecessors = []

for node in dag.topological_op_nodes():
    if node.name == 'ccx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-ccx'
            predecessors.append(predecessor)
            predecessor = list(dag.predecessors(node))[2].name + '-ccx'
            predecessors.append(predecessor)
        except:
            predecessor = 'Initial state-ccx'
            predecessors.append(predecessor)
    elif node.name == 'cx' or node.name == 'ch' or node.name == 'rcx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-cx'
        except:
            predecessor = 'Initial state-cx'
        predecessors.append(predecessor)
print(f'predecessors : {predecessors}')

# データの頻度を計算
counter = Counter(predecessors)
# データと頻度を分解
labels, values = zip(*counter.items())
# 棒グラフを描画
plt.bar(labels, values)
# 軸ラベルとタイトルを設定
plt.xlabel('predecessor gate')
plt.ylabel('Frequency')
plt.title('Histogram of predecessor gate')
# x軸のラベルを90度回転
plt.xticks(rotation=90)
# グラフを表示
plt.show()

In [ ]:
#############################
##### After remove_controlled_operations(qc3) #####
#############################
print('qc3 : After remove_controlled_operations')
# qc3.draw('mpl',fold = 100, initial_state = True, style = style)

In [ ]:
dag = circuit_to_dag(qc3)
predecessors = []

for node in dag.topological_op_nodes():
    if node.name == 'ccx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-ccx'
            predecessors.append(predecessor)
            predecessor = list(dag.predecessors(node))[2].name + '-ccx'
            predecessors.append(predecessor)
        except:
            predecessor = 'Initial state-ccx'
            predecessors.append(predecessor)
    elif node.name == 'cx' or node.name == 'ch' or node.name == 'rcx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-cx'
        except:
            predecessor = 'Initial state-cx'
        predecessors.append(predecessor)
print(f'predecessors : {predecessors}')

# データの頻度を計算
counter = Counter(predecessors)
# データと頻度を分解
labels, values = zip(*counter.items())
# 棒グラフを描画
plt.bar(labels, values)
# 軸ラベルとタイトルを設定
plt.xlabel('predecessor gate')
plt.ylabel('Frequency')
plt.title('Histogram of predecessor gate')
# x軸のラベルを90度回転
plt.xticks(rotation=90)
# グラフを表示
plt.show()

In [ ]:
#############################
##### After remove_adjacent(qc4) #####
#############################
print('qc4 : After remove_adjacent')
# qc4.draw('mpl',fold = 100, initial_state = True, style = style)

In [ ]:
dag = circuit_to_dag(qc4)
predecessors = []

for node in dag.topological_op_nodes():
    if node.name == 'ccx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-ccx'
            predecessors.append(predecessor)
            predecessor = list(dag.predecessors(node))[2].name + '-ccx'
            predecessors.append(predecessor)
        except:
            predecessor = 'Initial state-ccx'
            predecessors.append(predecessor)
    elif node.name == 'cx' or node.name == 'ch' or node.name == 'rcx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-cx'
        except:
            predecessor = 'Initial state-cx'
        predecessors.append(predecessor)
print(f'predecessors : {predecessors}')

# データの頻度を計算
counter = Counter(predecessors)
# データと頻度を分解
labels, values = zip(*counter.items())
# 棒グラフを描画
plt.bar(labels, values)
# 軸ラベルとタイトルを設定
plt.xlabel('predecessor gate')
plt.ylabel('Frequency')
plt.title('Histogram of predecessor gate')
# x軸のラベルを90度回転
plt.xticks(rotation=90)
# グラフを表示
plt.show()

In [ ]:
############################
##### After AQCEL(qc5) #####
############################
qc5 = result_qc
print('After AQCEL(qc5) : After remove_qubits')
# result_qc.draw('mpl',fold = 100, initial_state = True, style = style)

In [ ]:
dag = circuit_to_dag(result_qc)
predecessors = []

for node in dag.topological_op_nodes():
    if node.name == 'ccx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-ccx'
            predecessors.append(predecessor)
            predecessor = list(dag.predecessors(node))[2].name + '-ccx'
            predecessors.append(predecessor)
        except:
            predecessor = 'Initial state-ccx'
            predecessors.append(predecessor)
    elif node.name == 'cx' or node.name == 'ch' or node.name == 'rcx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-cx'
        except:
            predecessor = 'Initial state-cx'
        predecessors.append(predecessor)
print(f'predecessors : {predecessors}')

# データの頻度を計算
counter = Counter(predecessors)
labels, values = zip(*counter.items())

plt.bar(labels, values)
plt.xlabel('predecessor gate')
plt.ylabel('Frequency')
plt.title('Histogram of predecessor gate')
plt.xticks(rotation=90)
plt.show()

In [ ]:
############################
##### After transpiled(qc6) #####
############################
print('qc6 : After transpiled')
# qc6.draw('mpl',fold = 100, initial_state = True, style = style)

In [ ]:
dag = circuit_to_dag(qc6)
predecessors = []

for node in dag.topological_op_nodes():
    if node.name == 'ccx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-ccx'
            predecessors.append(predecessor)
            predecessor = list(dag.predecessors(node))[2].name + '-ccx'
            predecessors.append(predecessor)
        except:
            predecessor = 'Initial state-ccx'
            predecessors.append(predecessor)
    elif node.name == 'cx' or node.name == 'ch' or node.name == 'rcx':
        try:
            predecessor = list(dag.predecessors(node))[1].name + '-cx'
        except:
            predecessor = 'Initial state-cx'
        predecessors.append(predecessor)
print(f'predecessors : {predecessors}')

# データの頻度を計算
counter = Counter(predecessors)
labels, values = zip(*counter.items())
plt.bar(labels, values)
plt.xlabel('predecessor gate')
plt.ylabel('Frequency')
plt.title('Histogram of predecessor gate')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# aqcel_results_cc = [optimized_qc, transpiled_qc, depth_ccx_ori, depth_ccx_to_cx, depth_ccx_to_x, depth_ccx_to_none, depth_cx_ori, depth_cx_to_x, depth_cx_to_none]

##################################################################

###########################
##### Define depth_ccx_to_cx #####
###########################
depth_ccx_to_cx    =aqcel_results_cc[3]
print(f'depth_ccx_to_cx : {depth_ccx_to_cx}')
print(f'length of depth_ccx_to_cx : {len(depth_ccx_to_cx)}')
print('')



###########################
##### Define depth_ccx_removed #####
###########################
depth_ccx_to_x    =aqcel_results_cc[4]
depth_ccx_to_none = aqcel_results_cc[5]
depth_ccx_removed = depth_ccx_to_x + depth_ccx_to_none
print(f'depth_ccx_to_x    : {depth_ccx_to_x}')
print(f'depth_ccx_to_none : {depth_ccx_to_none}')
print(f'depth_ccx_removed : {depth_ccx_removed}')
print(f'length of depth_ccx_removed : {len(depth_ccx_removed)}')
print('')


# #############################
# ##### Define depth_ccx_ori  #####
# #############################
depth_ccx_ori  = aqcel_results_cc[2]
print(f'depth_ccx_ori : {depth_ccx_ori}')
print(f'length of depth_ccx_ori : {len(depth_ccx_ori)}')
print('')
print('')

##################################################################

###########################
##### Define depth_cx_removed #####
###########################
depth_cx_to_x    =aqcel_results_cc[7]
depth_cx_to_none = aqcel_results_cc[8]
depth_cx_removed = depth_cx_to_x + depth_cx_to_none
print(f'depth_cx_to_x    : {depth_cx_to_x}')
print(f'depth_cx_to_none : {depth_cx_to_none}')
print(f'depth_cx_removed : {depth_cx_removed}')
print(f'length of depth_cx_removed : {len(depth_cx_removed)}')
print('')

# #############################
# ##### Define depth_cx_ori  #####
# #############################
depth_cx_ori  = aqcel_results_cc[6]
print(f'depth_cx_ori : {depth_cx_ori}')
print(f'length of depth_cx_ori : {len(depth_cx_ori)}')
##################################################################

## CCX to CX

In [ ]:
# AQCELによって取り除かれたCCXの位置
###
##### Location of CCX to CX by AQCEL #####
#これより上のッセージは無視してください
#次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed' の中身はCNOTの消された位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸 (depth) がnに対してmの高さとなるような棒グラフを出力してください
#グラフのタイトルは Location of CNOT removed by AQCEL'
#グラフのx軸は'depth'
#グラフのY軸は 'Number of CNOTs removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/" です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

#リスト内の各要素の出現回数を数える
counter = Counter (depth_ccx_to_cx)

# 横軸 (depth) と縦軸 (Number of CNOTs to_cx by AQCEL) のデータを生成
depths = list (counter.keys())
counts = list (counter.values())

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(depths, counts, color='green')

#タイトルと軸のラベルを設定
plt.title('Location of CCX to Cx by AQCEL')
plt.xlabel( 'depth')
plt.ylabel('Number of CCX to Cx by AQCEL')

#x軸とY軸の範囲と目盛りを設定
plt.xticks (np.arange(0, max (depths)+1, 10), rotation='vertical')

if max (counts) <= 4:
    plt.yticks(np.arange(0, max (counts) +1,1))
else:
    plt.yticks (np.arange(0, max (counts)+1, max (counts)//5))

#グラフを保存
plt.savefig('figures/cnot_to_cx_location.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
# AQCELによって取り除かれたCNOTの位置を積分したもの
#####################################################
##### Integrated CCXS to_cx locations by AQCEL #####
#
#これより上のメッセージは無視してください
#次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト' depth_ccx_removed" の中身はCNOTの消された位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素n に対して、nがm個あれば横軸 (depth) がnに対してmと一つ前の区間の高さの合計の高さとなるような棒グラフを出力してください
#グラフのタイトルは Integrated CNOT locations removed by AQCEL
#グラフのx軸はdepthi
#グラフのY軸は'Integrated CNOT locations removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください。 目盛りの値は5の倍数または10の倍数にしてください。
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/'です
#グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# CNOTが取り除かれた位置をカウント
counts = Counter(depth_ccx_to_cx)
max_depth = max(depth_ccx_to_cx)

#x軸とy軸のデータを生成

x_ccx_to_cx = list(range (max_depth+1))
y = [counts.get(i, 0) for i in x_ccx_to_cx]

#積分 (累積和)を計算
integrated_y_ccx_to_cx = np.cumsum(y)
print("x_ccx_to_cx:", x_ccx_to_cx)
print("integrated_y_ccx_to_cx:", integrated_y_ccx_to_cx)

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(x_ccx_to_cx, integrated_y_ccx_to_cx, color= 'green')

# タイトルと軸のラベルを設定
plt.title('Integrated CCX to Cx locations by AQCEL')
plt.xlabel('depth')
plt.ylabel('Integrated CCX to Cx by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks (np.arange(0, max (x_ccx_to_cx)+1, 10), rotation='vertical')
plt.yticks (np.linspace(0, max(integrated_y_ccx_to_cx)+1,5))

#グラフを保存
plt.savefig('figures/integrated_cnot_locations_to_cx_by_aqcel.png', bbox_inches='tight')

#グラフを表示
plt.show()

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
M=16

#最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_ccx_to_cx)
intervals = np.linspace(0, max_val, M + 1 ).astype (int)
# 各区間での要素のカウントを保存するための空リストを作成

counts = []
# 各区間の要素をカウント
for i in range (len (intervals)-1):
    count = sum (intervals[i] <= val < intervals [i+1] for val in depth_ccx_to_cx)
    if i == len(intervals) -2: # 最後の区間では終点も含む
        count += depth_ccx_to_cx.count(intervals[i+1])
    counts.append (count)

# 区間ラベルを作成
interval_labels = [f"{intervals[i]}-{intervals [i+1]}" for i in range (len (intervals) -1)]

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CCX to Cx by AQCEL per section when divided into 16 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CCX to Cx')

#x軸とY軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks(np.arange(0, max (counts) +1,1))

#グラフを保存
plt.savefig('figures/cnot_to_cx_sections.png', bbox_inches='tight')

# グラフを表示
plt.show ( )

In [ ]:
# 4区間に分けた時の区間ごとのAQCELにより取り除かれたCNOTの数
######################
##### Number of CCX to Cx by AQCEL per section when divided into 4 sections #####
########################
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed' の中身はCNOTの消された位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、 区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、最後の区間では最後の値も含むようにしてください。
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。 個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルはNumber of CNOTs removed by AQCEL per section when divided into 4 sections'
# グラフのx軸は' Segment of depth'
# グラフのy軸は' Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/" です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください

import matplotlib.pyplot as plt
import numpy as np

M=4

# 最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_ccx_to_cx)
intervals = np.linspace(0, max_val, M +1).astype(int)

# 各区間での要素のカウントを保存するための空リストを作成
counts = []

# 各区間の要素をカウント
for i in range (len (intervals)-1):
    count = sum (intervals [i] <= val < intervals [i+1] for val in depth_ccx_to_cx)
    if i == len (intervals)-2: # 最後の区間では終点も含む
        count += depth_ccx_to_cx.count (intervals [i+1])
    counts.append (count)

# 区間ラベルを作成
interval_labels = [f"{intervals[i]} - {intervals [i+1]}" for i in range (len (intervals) -1)]

# グラフを描画
plt.figure (figsize= (10,6))
plt.bar (interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title ('Number of CCX to Cx by AQCEL per section when divided into 4 sections')
plt.xlabel ('Segment of depth')
plt.ylabel('Total number of CCX to Cx')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks (np.arange(0, max(counts) +1, 1))
plt.savefig('figures/cnot_to_cx_sections.png', bbox_inches = 'tight')
plt.show()

## CCX removed

In [ ]:
# AQCELによって取り除かれたCCXの位置
##############
##### Location of CCXS removed by AQCEL #####
##########################
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト"depth_ccx_removed' の中身はCNOTの消された位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸 (depth) がnに対してmの高さとなるような棒グラフを出力してください
# グラフのタイトルは"Location of CNOT removed by AQCEL'
#グラフのx軸は'depth'
#グラフのy軸は 'Number of CNOTs removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/' です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。


# リスト内の各要素の出現回数を数える
counter = Counter(depth_ccx_removed)

# 横軸 (depth) と縦軸 (Number of CNOT's removed by AQCEL) のデータを生成

depths = list (counter.keys())
counts = list (counter.values())

# グラフを描画
plt.figure (figsize= (10,6))
plt.bar (depths, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Location of CCXs removed by AQCEL')
plt.xlabel('depth')
plt.ylabel('Number of CCXS removed by AQCEL')

#x軸とY軸の範囲と目盛りを設定
plt.xticks (np.arange(0, max (depths)+1, 10), rotation='vertical')

if max (counts) <= 4:
    plt.yticks(np.arange(0, max (counts)+1,1))
else:
    plt.yticks(np.arange(0, max (counts) +1, max(counts)//5))

#グラフを保存
plt.savefig('figures/cnot_removed_location.png', bbox_inches='tight')
# グラフを表示
plt.show()

In [ ]:
# AQCELによって取り除かれたCNOTの位置を積分したもの
#################
##### Integrated CCXS locations removed by AQCEL #####
###################################
# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed の中身はCNOTの消された位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸 (depth) がn に対してmと一つ前の区間の高さの合計の高さとなるような棒グラフを出力してください
# グラフのタイトルは
#グラフのx軸は dentegrated CNOT Locations removed by AQCEL'
# グラフのy軸は Integrated CNOT locations removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください。 目盛りの値は5の倍数または10の倍数にしてください。
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所はfigures/' です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# CNOTが取り除かれた位置をカウント
counts = Counter(depth_ccx_removed)
max_depth = max(depth_ccx_removed)

#x軸とy軸のデータを生成
x = list(range (max_depth+1))
y = [counts.get (i, 0) for i in x]

#積分 (累積和)を計算
integrated_y_ls = np.cumsum(y)
print("x:", x)
print("Integrated y:", integrated_y_ls)

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(x, integrated_y_ls, color='green')

# タイトルと軸のラベルを設定
plt.title('Integrated CCXs locations removed by AQCEL')
plt.xlabel('depth')
plt.ylabel('Integrated CCXs locations removed by AQCEL')

# x軸とY軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(x)+1, 10), rotation='vertical')
plt.yticks(np.arange(0, max(integrated_y_ls)+10, 20))

# グラフを保存
plt.savefig('figures/integrated_cnot_locations_removed_by_aqcel.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
M = 16

# 最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_ccx_removed)
intervals = np.linspace(0, max_val, M + 1 ).astype (int)
# 各区間での要素のカウントを保存するための空リストを作成

counts = []
# 各区間の要素をカウント
for i in range (len (intervals)-1):
    count = sum (intervals[i] <= val < intervals [i+1] for val in depth_ccx_removed)
    if i == len(intervals) -2: # 最後の区間では終点も含む
        count += depth_ccx_removed.count(intervals [i+1])
    counts.append(count)

# 区間ラベルを作成
interval_labels = [f"{intervals[i]}-{intervals [i+1]}" for i in range (len (intervals)-1)]

#グラフを描画
plt.figure(figsize= (10,6))
plt.bar(interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title( 'Number of CCXs removed by AQCEL per section when divided into 16 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CCXs removed by AQCEL')

#x軸とY軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks(np.arange(0, max(counts)+1,5))

#グラフを保存
plt.savefig('figures/cnot_removed_sections.png', bbox_inches='tight')

#グラフを表示

plt.show()

In [ ]:
# 4区間に分けた時の区間ごとのAQCELにより取り除かれたCNOTの数
#########################################################################
##### Number of CCXS removed by AQCEL per section when divided into 4 sections #####
## #######################
#####
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed' の中身はCNOTの消された位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、 区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、最後の区間では最後の値も含むようにしてください。
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。 m個あればその区間の高さがmとなるような棒グラフを出力してください
#グラフのタイトルは Number of CNOTs removed by AQCEL per section when divided into 4 sections'
#グラフのx軸は' Segment of depth'
#グラフのy軸はTotal number of CNOTs'
#x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/" です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

M=4

# 最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_ccx_removed)
intervals = np.linspace(0, max_val, M +1).astype (int)

# 各区間での要素のカウントを保存するための空リストを作成
counts = []

# 各区間の要素をカウント
for i in range (len (intervals)-1):
    count = sum (intervals[i] <= val < intervals [i+1] for val in depth_ccx_removed)
    if i == len (intervals) -2: # 最後の区間では終点も含む
        count += depth_ccx_removed.count(intervals [i+1])
    counts.append (count)

# 区間ラベルを作成
interval_labels = [f" { intervals[i]}-{intervals [i+1]}" for i in range (len (intervals) -1)]

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title ('Number of CCXs removed by AQCEL per section when divided into 4 sections')
plt.xlabel ('Segment of depth')
plt.ylabel('Total number of CCXs removed by AQCEL')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks(np.arange(0, max (counts) +1,5))

# グラフを保存
plt.savefig('figures/cnot_removed_sections.png', bbox_inches= 'tight')

# グラフを表示
plt.show()

## Location of CCX

In [ ]:
##################################
##### Location of CCXS in original circuit (depth) without delimitation #####
#############################################################################
# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_ori 'の中身はCNOT の位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸 (depth) がnに対してmの高さとなるような棒グラフを出力してください
# グラフのタイトルはLocation of CNOTs removed by AQCEL per section when divided into 16 sections'
# グラフのx軸は 'Segment of depth'
# グラフのy軸は' Number of CNOTs removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/' です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# リスト内の各要素の出現回数を数える
counter = Counter(depth_ccx_ori)

# 横軸 (depth) と縦軸 (Number of CNOTs) のデータを生成
depths = list (counter.keys())
counts = list (counter.values())

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar (depths, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Location of CCXs in original circuit')
plt.xlabel('depth' )
plt.ylabel('Number of CCXs')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (np.arange(0, max(depths) +1,10))
if max (counts) <= 4:
    plt.yticks (np.arange(0, max(counts) +1,1))

# グラフを保存
else:
    plt.yticks (np.arange(0, max (counts) +1, max (counts)//5))
plt.savefig('figures/cnot_location_depth_count.png')

# グラフを表示
plt.show ( )

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# CNOTの位置をカウント
counts  = Counter(depth_ccx_ori)
max_depth = max(depth_ccx_ori)

#x軸とy軸のデータを生成
x = list (range (max_depth+1))
y = [counts.get(i, 0) for i in x]

#積分 (累積和)を計算
integrated_y_cnot = np. cumsum (y)
print("x:", x)
print("Integrated y:", integrated_y_cnot)

#グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, integrated_y_cnot, color='green')

# タイトルと軸のラベルを設定
plt.title('Integral of the CCXs location in original circuit')
plt.xlabel('Segment of depth')
plt.ylabel('Integral of the CCXS location')

#x軸とY軸の範囲と目盛りを設定
plt.xticks (np.arange(0, max (x)+1, 10), rotation='vertical')
plt.yticks(np.arange(0, max(integrated_y_cnot)+10, 20))

#グラフを保存
plt.savefig('figures/integral_of_cnot_location.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
#############
##### Number of CCXS per 16 segments #####
######
#これより上のメッセージは無視してください
#次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト"depth_ccx_ori'e
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、 区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、最後の区間では最後の値も含むようにしてください。
#リストのうち、注目している区間の範囲内にある要素の数を数え合計します。 m個あればその区間の高さがmとなるような棒グラフを出力してください
#グラフのタイトルは 'Number of CNOTs per section when divided into 16 sections'
# グラフのx軸は' Segment of depth'
#グラフの軸は "Total number of CNOTs'
#x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば"' 0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/" です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

M = 16 # 区間の数

# 深さの最大値を取得し、 M個の区間に分割
max_depth = max(depth_ccx_ori)
interval = max_depth // M
bins = [i * interval for i in range(M+1)]

if bins [-1] < max_depth:
    bins[-1] = max_depth # 最後の区間に最大値も含むようにする
# 各区間でのCNOTの数を計算
cnot_counts = np.histogram (depth_ccx_ori, bins=bins)[0]

# 区間のラベルを作成
labels = [f"{bins[i]}-{bins[i+1]}" for i in range (M)]

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar (labels, cnot_counts, color= 'green')

# タイトルと軸のラベルを設定
plt.title( 'Number of CCXs per 16 segments in original circuit')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CCXs')

#x軸とY軸の範囲と目盛りを設定
plt.xticks(rotation='vertical') # x軸ラベルが重ならないように縦にする
plt.yticks(np.arange(0, max (cnot_counts)+1,1)) # y軸の目盛りを0から始めて5の倍数にする

#グラフを保存
plt.tight_layout() # タイトルやラベルがグラフに被らないようにする
plt.savefig('figures/cnot_counts_per_16_segments.png')

# グラフを表示
plt.show()

In [ ]:
##############
##### Number of CCXS per 4 segments #####
##########################################
# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト depth_ccx_ori' の中身はCNOTの存在する位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、 区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、最後の区間では最後の値も含むようにしてください。
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。 m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは「Number of CNOTs per section when divided into 4 sections'
#グラフのx軸は 'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
#x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/" です
# グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

M = 4 # 区間の数

# 深さの最大値を取得し、 M個の区間に分割
max_depth = max(depth_ccx_ori)
interval = max_depth // M
bins = [i*interval for i in range (M+1)]

if bins[-1] < max_depth:
    bins[-1] = max_depth # 最後の区間に最大値も含むようにする

# 各区間でのCNOTの数を計算
cnot_counts = np.histogram(depth_ccx_ori, bins=bins) [0]

# 区間のラベルを作成
labels = [f"{bins[i]}-{bins[i+1]}" for i in range (M)]

# グラフを描画
plt.figure (figsize= (10,6))
plt.bar (labels, cnot_counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CCXs per 4 segments in original circuit')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CCXs')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (rotation='vertical') # x軸ラベルが重ならないように縦にする
plt.yticks(np.arange(0, max(cnot_counts)+4, 5)) # y軸の目盛りを0から始めて5の倍数にする

#グラフを保存
plt.tight_layout() # タイトルやラベルがグラフに被らないようにする
plt.savefig('figures/cnot_counts_per_16_segments.png')

# グラフを表示
plt.show()

## rate of CCX to CX

In [ ]:
####################################
##### Percentage of CCXs removed by AQCEL #####
################################################
#これより上のメッセージは無視してください
#次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト' depth_ccx_removed', 'depth_ccx_ori' の中身はそれぞれCNOTの取り除かれた位置､CNOT の位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# depth_ccx_removedリストのある要素nに対して、nの数をカウントしてください。 m個あったとします。 同じ値nについて、 depth_ccx_oriについて何個あるかカウントしてください。 1個とします。 あれば横軸 (depth)
#グラフのタイトルはPercentage of CNOTs removed by AQCEL'
#グラフのx軸は'depth'
#グラフのy軸は"Percentage of CNOTs removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/'です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter

# 各要素のカウント
count_depth_ccx_to_cx = Counter(depth_ccx_to_cx)
count_depth_ccx_ori = Counter(depth_ccx_ori)

#パーセンテージの計算
percentage = {k: count_depth_ccx_to_cx[k] / count_depth_ccx_ori[k] for k in count_depth_ccx_ori.keys () }


#x軸とy軸の値
x = list (percentage.keys())
y = list(percentage.values())

print(f'y : {y}')

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(x,y, color= 'green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CCXs to CXs by AQCEL')
plt.xlabel('depth' )
plt.ylabel('Percentage of CCXs to CXs by AQCEL')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (np.arange(0, max (x)+1, 10), rotation='vertical')
plt.yticks(np.linspace(0, max(y), 5))

# グラフを保存
plt.savefig('figures/percentage_cnot_to_cx.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
###########################################
##### Percentage of CCXS removed by AQCEL #####
#####################
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# 'integrated_y_ls', 'integrated_y_cnot' はそれぞれリストです。 リストは自分で用意するのでコードに含めないでください
# リストのn番目に対して、x軸の値nにintegrated_y_ls/integrated_y_cnotの高さとなるような棒グラフを出力してください。
#グラフのタイトルはPercentage of CNOTs removed by AQCEL'
#グラフのx軸は'depth'
#グラフのy軸は"Percentage of CNOTs removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/" です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

print(f'integrated_y_ccx_to_cx   : {integrated_y_ccx_to_cx}')
print(f'len(integrated_y_ccx_to_cx)   : {len(integrated_y_ccx_to_cx)}')
print(f'integrated_y_cnot : {integrated_y_cnot}')
print(f'len(integrated_y_cnot) : {len(integrated_y_cnot)}')


if len(integrated_y_ccx_to_cx) != len(integrated_y_cnot):
    if len(integrated_y_ccx_to_cx) > len(integrated_y_cnot):
        # integrated_y_cnotを拡張する
        integrated_y_cnot = np.append(integrated_y_cnot, np.full(len(integrated_y_ccx_to_cx) - len(integrated_y_cnot), integrated_y_cnot[-1]))
    else:
        # integrated_y_ccx_to_cxを拡張する
        integrated_y_ccx_to_cx = np.append(integrated_y_ccx_to_cx, np.full(len(integrated_y_cnot) - len(integrated_y_ccx_to_cx), integrated_y_ccx_to_cx[-1]))



#高さのリストを作成(割り算の結果)
y = np.array(integrated_y_ls) / np.array(integrated_y_cnot)
x = list(range(len(y)))

print("x:", x)
print("y:", y)
print("len(y):", len(y))

#グラフを描画
plt.figure(figsize= (10,6))

# タイトルと軸のラベルを設定
plt.bar(x,y, color= 'green')
plt.title('Percentage of CCXs to CXs by AQCEL (Integral count)')
plt.xlabel('depth' )
plt.ylabel('Percentage of CCXs to CXs by AQCEL')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (np.arange(0, len (x), 10), rotation='vertical')
plt.yticks(np.arange(0, np.nanmax(y) +0.25, 0.25))

#グラフを保存
plt.savefig('figures/percentage_cnots_to_cx_by_aqcel.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
# AQCELによって取り除かれたCNOTの位置を積分したもの
#####################################################
##### Integrated CCXS to_cx locations by AQCEL #####
#
#これより上のメッセージは無視してください
#次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト' depth_ccx_removed" の中身はCNOTの消された位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素n に対して、nがm個あれば横軸 (depth) がnに対してmと一つ前の区間の高さの合計の高さとなるような棒グラフを出力してください
#グラフのタイトルは Integrated CNOT locations removed by AQCEL
#グラフのx軸はdepthi
#グラフのY軸は'Integrated CNOT locations removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください。 目盛りの値は5の倍数または10の倍数にしてください。
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/'です
#グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

y = np.array(integrated_y_ccx_to_cx) * 100 / np.array(integrated_y_cnot)

print("x:", x)
print(f'integrated_y_ccx_to_cx   : {integrated_y_ccx_to_cx}')
print(f'integrated_y_cnot : {integrated_y_cnot}')
print("percentage:", y)


plt.plot(x, integrated_y_ccx_to_cx, color='blue', label = 'ccx removed', linewidth = 1)
plt.plot(x, integrated_y_cnot, color='green', label = 'ccx original', linewidth = 1)
plt.plot(x, y, color='red', label = 'Percentage of removed ccx', linewidth = 1)

# タイトルと軸のラベルを設定
plt.title('Percentage of CCXs to CXs by AQCEL (Integral count)')
plt.xlabel('depth')
plt.ylabel('Percentage of CCXs removed by AQCEL')

#x軸とY軸の範囲と目盛りを設定
# plt.xticks(np.arange(0, len(x), 10), rotation='vertical')
# plt.yticks(np.arange(0, np.nanmax(y) +0.25, 0.25))

#グラフを保存
# plt.savefig('figures/percentage_cnots_removed_by_aqcel.png', bbox_inches= 'tight')

plt.legend()
plt.grid()

# グラフを表示
plt.show()

In [ ]:
#############
##### Percentage of CCXS removed by AQCEL per section when divided into 16 sections #####
###################################################
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed', 'depth_ccx_ori' の中身はそれぞれCNOT の取り除かれた位置 CNOT の位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、最後の区間では最後の値も含むようにしてください。 ここ
# depth_ccx_removedリストのうち、注目している区間の範囲内にある要素の数をカウントし合計してください。 m個あったとします。 同じ区間について、今度はdepth_ccx_oriに何個あるかカウントし合計してください。 1個とします。 あれば
#グラフのタイトルはPercentage of CNOT removed by AQCEL per section when divided into 16 sections'
#グラフのx軸は' Segment of depth'
# グラフのy軸はPercentage of CNOT removed by AQCEL per section'
#x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/" です
#グラフの色は緑にしてください
#debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np
import math

# 区間数
M=16

# 最大値の取得
max_depth = max(max(depth_ccx_to_cx), max(depth_ccx_ori) )

# 区間の境界値を計算
bins = np.linspace(0, max_depth, M+1)

# 区間に分けて要素の数をカウント
counts_depth_ccx_to_cx, _ = np.histogram(depth_ccx_to_cx, bins=bins)
counts_depth_ccx_ori, _ = np.histogram (depth_ccx_ori, bins=bins)

#パーセンテージの計算
percentage = [0 if c_cnot == 0 else c_ls/c_cnot for c_ls, c_cnot in zip (counts_depth_ccx_to_cx, counts_depth_ccx_ori)]

# x軸とy軸の値
x = [f'{math.floor (bins[i])}-{math.ceil (bins [i+1])}' for i in range (M)]
y = percentage

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(x,y, color= 'green')
# タイトルと軸のラベルを設定

plt.title ('Percentage of CCXs to CXs by AQCEL per section when divided into 16 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Percentage of CCXs to CXs by AQCEL per section')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks (np.arange(0, np.nanmax(y)+0.25, 0.25))

#グラフを保存
plt.savefig('figures/percentage_ccx_to_cx_per_section.png', bbox_inches='tight')

# グラフを表示
plt.show()
      

In [ ]:
####################################################################
##### Percentage of CCXS removed by AQCEL per section when divided into 4 sections #####
########################################################################################
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed', 'depth_ccx_ori' の中身はそれぞれCNOTの取り除かれた位置､CNOTの位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、 区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、最後の区間では最後の値も含むようにしてくださ
# depth_ccx_removedリストのうち、注目している区間の範囲内にある要素の数をカウントし合計してください。 m個あったとします。 同じ区間について、 今度はdepth_ccx_oriに何個あるかカウントし合計してください。 1個とします。
#グラフのタイトルはPercentage of CNOT removed by AQCEL per section when divided into 4 sections'
#グラフのx軸はSegment of depth'
#グラフのy軸は' Percentage of CNOT removed by AQCEL per section'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/" です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np
import math

# 区間数
M=4

# 最大値の取得
max_depth = max(max(depth_ccx_to_cx), max(depth_ccx_ori))

# 区間の境界値を計算
bins = np.linspace(0, max_depth, M+1)

# 区間に分けて要素の数をカウント
counts_depth_ccx_to_cx,_ = np.histogram (depth_ccx_to_cx, bins=bins)
counts_depth_ccx_ori,_ = np.histogram (depth_ccx_ori, bins=bins)

#パーセンテージの計算
percentage = [0 if c_cnot == 0 else c_ls/c_cnot for c_ls, c_cnot in zip (counts_depth_ccx_to_cx, counts_depth_ccx_ori)]

#x軸とy軸の値
x = [f'{math.floor(bins[i])}-{math.ceil(bins[i+1])}' for i in range(M)]
y = percentage

#グラフを描画
plt.figure(figsize= (10,6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CCXs to CXs by AQCEL per section when divided into 4 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Percentage of CCXs to CXs by AQCEL per section')

#x軸とy軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks(np.arange(0, np.nanmax(y)+0.25, 0.1))

#グラフを保存
plt.savefig('figures/percentage_cnot_to_cx_per_section.png', bbox_inches='tight')

# グラフを表示
plt.show()

## rate of CCX removed completely

In [ ]:
###############################
##### Percentage of CCXS removed by AQCEL #####
################################################
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python で matplotlibを用いてください。 axは使わずplt. を用いてください
# リスト 'depth_ccx_removed', 'depth_ccx_ori' の中身はそれぞれCNOTの取り除かれた位置 CNOT の位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# depth_ccx_removedリストのある要素nに対して、nの数をカウントしてください。 m個あったとします。 同じ値nについて、 depth_ccx_oriについて何個あるかカウントしてください。 1個とします。 あれば横軸 (depth) がnに対して
#グラフのタイトルはPercentage of CNOTs removed by AQCEL'
# グラフのx軸は'depth'
#グラフのy軸は' Percentage of CNOTs removed by AQCEL'
#x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/'です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter

# 各要素のカウント
count_depth_ccx_removed = Counter(depth_ccx_removed)
count_depth_ccx_ori = Counter(depth_ccx_ori)

#パーセンテージの計算
percentage = {k: count_depth_ccx_removed[k] / count_depth_ccx_ori [k] for k in count_depth_ccx_ori.keys() }

#x軸とy軸の値
x = list (percentage.keys ())
y = list (percentage.values())

#グラフを描画
plt.figure (figsize=(10,6))
plt.bar(x,y, color= 'green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CCXs removed by AQCEL')
plt.xlabel('depth')
plt.ylabel('Percentage of CCXs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks (np.arange(0, max (x)+1, 10), rotation='vertical')
plt.yticks (np.linspace(0, max(y), 5))

#グラフを保存
plt.savefig('figures/percentage_cnot_removed.png', bbox_inches='tight')

#グラフを表示
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print(f'integrated_y_ls   : {integrated_y_ls}')
print(f'len(integrated_y_ls)   : {len(integrated_y_ls)}')
print(f'integrated_y_cnot : {integrated_y_cnot}')
print(f'len(integrated_y_cnot) : {len(integrated_y_cnot)}')

print(f'integrated_y_ls   : {integrated_y_ls}')
print(f'integrated_y_cnot : {integrated_y_cnot}')

#高さのリストを作成 ( 割り算の結果)
y = np.array (integrated_y_ls) / np.array (integrated_y_cnot)
x = list(range (len(y)))
print("x:", x)
print("y:", y)

#グラフを描画
plt.figure (figsize= (10,6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CCXs removed by AQCEL (Integral count)')
plt.xlabel('depth')
plt.ylabel('Percentage of CCXs removed by AQCEL')

#x軸とY軸の範囲と目盛りを設定
plt.xticks(np.arange(0, len(x), 10), rotation='vertical')
plt.yticks(np.arange(0, np.nanmax(y) +0.25, 0.25))

#グラフを保存
plt.savefig('figures/percentage_cnots_removed_by_aqcel.png', bbox_inches= 'tight')

# グラフを表示
plt.show()

In [ ]:
# AQCELによって取り除かれたCNOTの位置を積分したもの
#################
##### Integrated CCXS locations removed by AQCEL #####
###################################
# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed の中身はCNOTの消された位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸 (depth) がn に対してmと一つ前の区間の高さの合計の高さとなるような棒グラフを出力してください
# グラフのタイトルは
#グラフのx軸は dentegrated CNOT Locations removed by AQCEL'
# グラフのy軸は Integrated CNOT locations removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください。 目盛りの値は5の倍数または10の倍数にしてください。
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所はfigures/' です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

y = np.array (integrated_y_ls) *100 / np.array (integrated_y_cnot)

print("x:", x)
print("Integrated y:", integrated_y_ls)
print("Integrated y:", integrated_y_cnot)
print("percentage:", y)

#グラフを描画
plt.plot(x, integrated_y_ls, color='blue', label = 'ccx removed', linewidth = 1)
plt.plot(x, integrated_y_cnot, color='green', label = 'ccx original', linewidth = 1)
plt.plot(x, y, color='red', label = 'Percentage of removed ccx', linewidth = 1)

# # NaNの位置にテキストを追加
# for i, val in enumerate(y):
#     if np.isnan(val):
#         plt.text(x[i], 0, 'NaN', ha='center', va='center', color='red')

# タイトルと軸のラベルを設定
plt.title('Percentage of CCXs removed by AQCEL (Integral count)')
plt.xlabel('depth')
plt.ylabel('Percentage of CCXs removed by AQCEL')
# plt.xticks(np.arange(0, len(x), 10), rotation='vertical')
# plt.yticks(np.arange(0, np.nanmax(y) +0.25, 0.25))
# plt.savefig('figures/percentage_cnots_removed_by_aqcel.png', bbox_inches= 'tight')
plt.legend(loc='lower right')
plt.grid()
plt.show()


In [ ]:
####################################################################*********************
##### Percentage of CCXS removed by AQCEL per section when divided into 16 sections #####
#########################################################################################
#これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。 ax.は使わずplt. を用いてください
# リスト' depth_ccx_removed', 'depth_ccx_ori' の中身はそれぞれCNOTの取り除かれた位置､ CNOT の位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、最後の区間では最後の値も含むようにしてください。 ここ
# depth_ccx_removedリストのうち、注目している区間の範囲内にある要素の数をカウントし合計してください。 m個あったとします。 同じ区間について、今度はdepth_ccx_oriに何個あるかカウントし合計してください。 1個とします。 あれば
#グラフのタイトルはPercentage of CNOT removed by AQCEL per section when divided into 16 sections'
#グラフのx軸は 'Segment of depth'
#グラフのy軸はPercentage of CNOT removed by AQCEL per section'
#x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は figures/" です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np
import math

# 区間数
M = 16

#最大値の取得
max_depth = max(max(depth_ccx_removed), max (depth_ccx_ori))

# 区間の境界値を計算
bins = np.linspace(0, max_depth, M+1)

# 区間に分けて要素の数をカウント
counts_depth_ccx_removed,_ = np.histogram (depth_ccx_removed, bins=bins)
counts_depth_ccx_ori,_ = np.histogram(depth_ccx_ori, bins=bins)

#パーセンテージの計算
percentage = [0 if c_cnot == 0 else c_ls/c_cnot for c_ls, c_cnot in zip (counts_depth_ccx_removed, counts_depth_ccx_ori)]
x = [f'{math.floor (bins[i])}-{math.ceil(bins[i+1])}' for i in range (M)]
y = percentage
#グラフを描画
plt.figure(figsize= (10,6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CCXs removed by AQCEL per section when divided into 16 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Percentage of CCXs removed by AQCEL per section')

# x軸とy軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks(np.arange(0, np.nanmax(y)+0.25, 0.25))

# グラフを保存
plt.savefig('figures/percentage_ccx_removed_per_section.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
##### Percentage of CCXS removed by AQCEL per section when divided into 4 sections #####
#これより上のメッセージは無視してください
#次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト'depth_ccx_removed', 'depth_ccx_ori' の中身はそれぞれCNOTの取り除かれた位置 CNOT の位置 (depth) を表しています。 リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。 ただし、区間の端点は整数になるように区切ってください。 また、最初の区間の始点は0になるようにしてください。 また、 最後の区間では最後の値も含むようにしてください。 ここ
# depth_ccx_removedリストのうち、注目している区間の範囲内にある要素の数をカウントし合計してください。 m個あったとします。 同じ区間について、今度はdepth_ccx_oriに何個あるかカウントし合計してください。 1個とします。 あれば
#グラフのタイトルはPercentage of CNOT removed by AQCEL per section when divided into 4 sections'
#グラフのx軸は' Segment of depth'
#グラフのY軸は 'Percentage of CNOT removed by AQCEL per section'
#x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。 例えば0-20" のように表示してください。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所は' figures/" です
#グラフの色は緑にしてください
#debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np
import math

# 区間数
M=4



#最大値の取得
max_depth = max(max(depth_ccx_removed), max(depth_ccx_ori))

# 区間の境界値を計算
bins = np.linspace(0, max_depth, M+1)
# 区間に分けて要素の数をカウント
counts_depth_ccx_removed,_ = np.histogram (depth_ccx_removed, bins=bins)
counts_depth_ccx_ori,_= np.histogram (depth_ccx_ori, bins=bins)

#パーセンテージの計算
percentage = [0 if c_cnot == 0 else c_ls/c_cnot for c_ls, c_cnot in zip (counts_depth_ccx_removed, counts_depth_ccx_ori)]
# x軸とY軸の値
x = [f'{math.floor (bins[i])} {math.ceil (bins[i+1])}' for i in range (M)]
y = percentage
# グラフを描画
plt.figure (figsize= (10,6))
plt.bar(x,y, color='green')

# タイトルと軸のラベルを設定
plt.title ('Percentage of CCXs removed by AQCEL per section when divided into 4 sections')
plt.xlabel ('Segment of depth')
plt.ylabel('Percentage of CCXs removed by AQCEL per section')

# x軸とy軸の範囲と目盛りを設定
plt.xticks (rotation='vertical')
plt.yticks (np.arange(0, np.nanmax(y) +0.25, 0.25))

# グラフを保存
plt.savefig('figures/percentage_cnot_removed_per_section.png', bbox_inches='tight')

# グラフを表示
plt.show()

## CX removed

In [ ]:
# AQCELによって取り除かれたCNOTの位置
#############################################
##### Location of CNOT removed by AQCEL #####
#############################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_removed'の中身はCNOTの消された位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸（depth)がnに対してmの高さとなるような棒グラフを出力してください
# グラフのタイトルは'Location of CNOT removed by AQCEL'
# グラフのx軸は'depth'
# グラフのy軸は'Number of CNOTs removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
from collections import Counter
import numpy as np


# リスト内の各要素の出現回数を数える
counter = Counter(depth_cx_removed)

# 横軸（depth）と縦軸（Number of CNOTs removed by AQCEL）のデータを生成
depths = list(counter.keys())
counts = list(counter.values())

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(depths, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Location of CNOTs removed by AQCEL')
plt.xlabel('depth')
plt.ylabel('Number of CNOTs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(depths)+1, 10), rotation='vertical')
if max(counts) <= 4:
    plt.yticks(np.arange(0, max(counts)+1, 1))
else:
    plt.yticks(np.arange(0, max(counts)+1, max(counts)//5))

# グラフを保存
plt.savefig('figures/cnot_removed_location.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
# AQCELによって取り除かれたCNOTの位置を積分したもの

######################################################
##### Integrated CNOT locations removed by AQCEL #####
######################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_removed'の中身はCNOTの消された位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸（depth)がnに対してmと一つ前の区間の高さの合計の高さとなるような棒グラフを出力してください
# グラフのタイトルは'Integrated CNOT locations removed by AQCEL'
# グラフのx軸は'depth'
# グラフのy軸は'Integrated CNOT locations removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください。目盛りの値は5の倍数または10の倍数にしてください。
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# depth_cx_removed = ...  # データの定義

# CNOTが取り除かれた位置をカウント
counts = Counter(depth_cx_removed)
max_depth = max(depth_cx_removed)

# x軸とy軸のデータを生成
x = list(range(max_depth+1))
y = [counts.get(i, 0) for i in x]

# 積分（累積和）を計算
integrated_y_ls = np.cumsum(y)

print("x:", x)
print("Integrated y:", integrated_y_ls)

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, integrated_y_ls, color='green')

# タイトルと軸のラベルを設定
plt.title('Integrated CNOTs locations removed by AQCEL')
plt.xlabel('depth')
plt.ylabel('Integrated CNOTs locations removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(x)+1, 10), rotation='vertical')
plt.yticks(np.arange(0, max(integrated_y_ls)+5, 5))

# グラフを保存
plt.savefig('figures/integrated_cnot_locations_removed_by_aqcel.png', bbox_inches='tight')

# グラフを表示
plt.show()


In [ ]:
# 16区間に分けた時の区間ごとのAQCELにより取り除かれたCNOTの数
######################################################################################
##### Number of CNOTs removed by AQCEL per section when divided into 16 sections #####
######################################################################################


# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_removed'の中身はCNOTの消された位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=16とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs removed by AQCEL per section when divided into 16 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

M = 16

# 最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_cx_removed)
intervals = np.linspace(0, max_val,M + 1  ).astype(int)

# 各区間での要素のカウントを保存するための空リストを作成
counts = []

# 各区間の要素をカウント
for i in range(len(intervals)-1):
    count = sum(intervals[i] <= val < intervals[i+1] for val in depth_cx_removed)
    if i == len(intervals)-2:  # 最後の区間では終点も含む
        count += depth_cx_removed.count(intervals[i+1])
    counts.append(count)

# 区間ラベルを作成
interval_labels = [f"{intervals[i]}-{intervals[i+1]}" for i in range(len(intervals)-1)]

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs removed by AQCEL per section when divided into 16 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')
plt.yticks(np.arange(0, max(counts)+1, 1))

# グラフを保存
plt.savefig('figures/cnot_removed_sections.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
# 4区間に分けた時の区間ごとのAQCELにより取り除かれたCNOTの数
#####################################################################################
##### Number of CNOTs removed by AQCEL per section when divided into 4 sections #####
#####################################################################################


# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_removed'の中身はCNOTの消された位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=4とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs removed by AQCEL per section when divided into 4 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。
# 
import matplotlib.pyplot as plt
import numpy as np

M = 4

# 最大値を取得し、16区間に分割するための境界を作成
max_val = max(depth_cx_removed)
intervals = np.linspace(0, max_val, M +1).astype(int)

# 各区間での要素のカウントを保存するための空リストを作成
counts = []

# 各区間の要素をカウント
for i in range(len(intervals)-1):
    count = sum(intervals[i] <= val < intervals[i+1] for val in depth_cx_removed)
    if i == len(intervals)-2:  # 最後の区間では終点も含む
        count += depth_cx_removed.count(intervals[i+1])
    counts.append(count)

# 区間ラベルを作成
interval_labels = [f"{intervals[i]}-{intervals[i+1]}" for i in range(len(intervals)-1)]

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(interval_labels, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs removed by AQCEL per section when divided into 4 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')
plt.yticks(np.arange(0, max(counts)+5, 2))

# グラフを保存
plt.savefig('figures/cnot_removed_sections.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
#############################################################################
##### Location of CNOT in original circuit (depth) without delimitation #####
#############################################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_ori 'の中身はCNOTの位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸（depth)がnに対してmの高さとなるような棒グラフを出力してください
# グラフのタイトルは'Location of CNOTs removed by AQCEL per section when divided into 16 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Number of CNOTs removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
from collections import Counter
import numpy as np


# リスト内の各要素の出現回数を数える
counter = Counter(depth_cx_ori)

# 横軸（depth）と縦軸（Number of CNOTs）のデータを生成
depths = list(counter.keys())
counts = list(counter.values())

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(depths, counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Location of CNOTs in original circuit')
plt.xlabel('depth')
plt.ylabel('Number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(depths)+1, 10))
if max(counts) <= 4:
    plt.yticks(np.arange(0, max(counts)+1, 1))
else:
    plt.yticks(np.arange(0, max(counts)+1, max(counts)//5))

# グラフを保存
plt.savefig('figures/cnot_location_depth_count.png')

# グラフを表示
plt.show()

In [ ]:
#########################################
##### Integral of the CNOT location #####
#########################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_ori'の中身はCNOTの位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸（depth)がnに対してmと一つ前の区間の高さの合計の高さとなるような棒グラフを出力してください
# グラフのタイトルは'Integral of the CNOT location'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Integral of the CNOT location'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください。目盛りの値は5の倍数または10の倍数にしてください。
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter
import numpy as np



# CNOTの位置をカウント
counts = Counter(depth_cx_ori)
max_depth = max(depth_cx_ori)

# x軸とy軸のデータを生成
x = list(range(max_depth+1))
y = [counts.get(i, 0) for i in x]

# 積分（累積和）を計算
integrated_y_cnot = np.cumsum(y)

print("x:", x)
print("Integrated y:", integrated_y_cnot)

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, integrated_y_cnot, color='green')

# タイトルと軸のラベルを設定
plt.title('Integral of the CNOTs location in original circuit')
plt.xlabel('Segment of depth')
plt.ylabel('Integral of the CNOT')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(x)+1, 10), rotation='vertical')
plt.yticks(np.arange(0, max(integrated_y_cnot)+10, 5))

# グラフを保存
plt.savefig('figures/integral_of_cnot_location.png', bbox_inches='tight')

# グラフを表示
plt.show()


In [ ]:
###########################################
##### Number of CNOTs per 16 segments #####
###########################################

# 

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_ori'の中身はCNOTの存在する位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=16とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs per section when divided into 16 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np

M = 16  # 区間の数

# 深さの最大値を取得し、M個の区間に分割
max_depth = max(depth_cx_ori)
interval = max_depth // M
bins = [i*interval for i in range(M+1)]
if bins[-1] < max_depth:
    bins[-1] = max_depth  # 最後の区間に最大値も含むようにする


# 各区間でのCNOTの数を計算
cnot_counts = np.histogram(depth_cx_ori, bins=bins)[0]

# 区間のラベルを作成
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(M)]

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(labels, cnot_counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs per 16 segments in original circuit')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')  # x軸ラベルが重ならないように縦にする
plt.yticks(np.arange(0, max(cnot_counts)+1, 1))  # y軸の目盛りを0から始めて5の倍数にする

# グラフを保存
plt.tight_layout()  # タイトルやラベルがグラフに被らないようにする
plt.savefig('figures/cnot_counts_per_16_segments.png')

# グラフを表示
plt.show()

In [ ]:
##########################################
##### Number of CNOTs per 4 segments #####
##########################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_ori'の中身はCNOTの存在する位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=4とします
# リストのうち、注目している区間の範囲内にある要素の数を数え合計します。m個あればその区間の高さがmとなるような棒グラフを出力してください
# グラフのタイトルは'Number of CNOTs per section when divided into 4 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Total number of CNOTs'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて5の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
import numpy as np

M = 4  # 区間の数

# 深さの最大値を取得し、M個の区間に分割
max_depth = max(depth_cx_ori)
interval = max_depth // M
bins = [i*interval for i in range(M+1)]
if bins[-1] < max_depth:
    bins[-1] = max_depth  # 最後の区間に最大値も含むようにする


# 各区間でのCNOTの数を計算
cnot_counts = np.histogram(depth_cx_ori, bins=bins)[0]

# 区間のラベルを作成
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(M)]

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(labels, cnot_counts, color='green')

# タイトルと軸のラベルを設定
plt.title('Number of CNOTs per 4 segments in original circuit')
plt.xlabel('Segment of depth')
plt.ylabel('Total number of CNOTs')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')  # x軸ラベルが重ならないように縦にする
plt.yticks(np.arange(0, max(cnot_counts)+1, 2))  # y軸の目盛りを0から始めて5の倍数にする

# グラフを保存
plt.tight_layout()  # タイトルやラベルがグラフに被らないようにする
plt.savefig('figures/cnot_counts_per_16_segments.png')

# グラフを表示
plt.show()

In [ ]:
################################################
##### Percentage of CNOTs removed by AQCEL #####
################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_removed', 'depth_cx_ori'の中身はそれぞれCNOTの取り除かれた位置、CNOTの位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depth_cx_removedリストのある要素nに対して、nの数をカウントしてください。m個あったとします。同じ値nについて、depth_cx_oriについて何個あるかカウントしてください。l個とします。あれば横軸（depth)がnに対して高さがm/lとなるような棒グラフを出力してください。ただし、nがdepth_cx_removedに含まれていない場合は高さは0としてください。
# グラフのタイトルは'Percentage of CNOTs removed by AQCEL'
# グラフのx軸は'depth'
# グラフのy軸は'Percentage of CNOTs removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
from collections import Counter


# 各要素のカウント
count_depth_cx_removed = Counter(depth_cx_removed)
count_depth_cx_ori = Counter(depth_cx_ori)

# パーセンテージの計算
percentage = {k: count_depth_cx_removed[k] / count_depth_cx_ori[k] for k in count_depth_cx_ori.keys()}

# x軸とy軸の値
x = list(percentage.keys())
y = list(percentage.values())

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CNOTs removed by AQCEL in original circuit')
plt.xlabel('depth')
plt.ylabel('Percentage of CNOTs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, max(x)+1, 10), rotation='vertical')
plt.yticks(np.linspace(0, max(y), 5))

# グラフを保存
plt.savefig('figures/percentage_cnot_removed.png', bbox_inches='tight')

# グラフを表示
plt.show()


In [ ]:
################################################
##### Percentage of CNOTs removed by AQCEL #####
################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# 'integrated_y_ls', 'integrated_y_cnot'はそれぞれリストです。リストは自分で用意するのでコードに含めないでください
# リストのn番目に対して、x軸の値nにintegrated_y_ls/integrated_y_cnotの高さとなるような棒グラフを出力してください。
# グラフのタイトルは'Percentage of CNOTs removed by AQCEL'
# グラフのx軸は'depth'
# グラフのy軸は'Percentage of CNOTs removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。


import matplotlib.pyplot as plt
import numpy as np

# integrated_y_ls = ...  # データの定義
# integrated_y_cnot = ...  # データの定義

# 高さのリストを作成（割り算の結果）
y = np.array(integrated_y_ls) / np.array(integrated_y_cnot)
x = list(range(len(y)))



print("x:", x)
print("y:", y)

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CNOTs removed by AQCEL (Integral Count)')
plt.xlabel('depth')
plt.ylabel('Percentage of CNOTs removed by AQCEL')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(np.arange(0, len(x), 10), rotation='vertical')
plt.yticks(np.arange(0, np.nanmax(y) + 0.25, 0.25))

# グラフを保存
plt.savefig('figures/percentage_cnots_removed_by_aqcel.png', bbox_inches='tight')

# グラフを表示
plt.show()


In [ ]:
# AQCELによって取り除かれたCNOTの位置を積分したもの
#################
##### Integrated CCXS locations removed by AQCEL #####
###################################
# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Python matplotlibを用いてください。 ax. は使わずplt. を用いてください
# リスト 'depth_ccx_removed の中身はCNOTの消された位置 (depth)を表しています。 リストは自分で用意するのでコードに含めないでください
# リストのある要素nに対して、nがm個あれば横軸 (depth) がn に対してmと一つ前の区間の高さの合計の高さとなるような棒グラフを出力してください
# グラフのタイトルは
#グラフのx軸は dentegrated CNOT Locations removed by AQCEL'
# グラフのy軸は Integrated CNOT locations removed by AQCEL'
# x軸の目盛りは0から始めて最大値まで10の倍数ごとに表示。 隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて最大値までおおよそ均等に5つ程度表示。 ただし縦軸の最大値が4以下の場合は見やすいように適切に目盛りを設定してください。 目盛りの値は5の倍数または10の倍数にしてください。
# 適切な名前をつけて保存してください。 ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。 保存場所はfigures/' です
#グラフの色は緑にしてください
# debugしやすいように、 代入した変数全てについてできるだけ細かくprint関数で出力してください。 また、 printされた値が何かわかるように簡単な説明も一緒にprintしてください。

y = np.array(integrated_y_ls) *100 / np.array(integrated_y_cnot)

print("x:", x)
print("Integrated y:", integrated_y_ls)
print("Integrated y:", integrated_y_cnot)
print("percentage:", y)

#グラフを描画
plt.plot(x, integrated_y_ls, color='blue', label = 'cx removed', linewidth = 1)
plt.plot(x, integrated_y_cnot, color='green', label = 'cx original', linewidth = 1)
plt.plot(x, y, color='red', label = 'Percentage of removed cx', linewidth = 1)

# # NaNの位置にテキストを追加
# for i, val in enumerate(y):
#     if np.isnan(val):
#         plt.text(x[i], 0, 'NaN', ha='center', va='center', color='red')

# タイトルと軸のラベルを設定
plt.title('Percentage of CNOTs removed by AQCEL (Integral count)')
plt.xlabel('depth')
plt.ylabel('Percentage of CNOTs removed by AQCEL')
# plt.xticks(np.arange(0, len(x), 10), rotation='vertical')
# plt.yticks(np.arange(0, np.nanmax(y) +0.25, 0.25))
# plt.savefig('figures/percentage_cnots_removed_by_aqcel.png', bbox_inches= 'tight')
plt.legend()
plt.grid()
plt.show()


In [ ]:
#########################################################################################
##### Percentage of CNOT removed by AQCEL per section when divided into 16 sections #####
#########################################################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_removed', 'depth_cx_ori'の中身はそれぞれCNOTの取り除かれた位置、CNOTの位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=16とします
# depth_cx_removedリストのうち、注目している区間の範囲内にある要素の数をカウントし合計してください。m個あったとします。同じ区間について、今度はdepth_cx_oriに何個あるかカウントし合計してください。l個とします。あれば横軸（depth)がnに対して高さがm/lとなるような棒グラフを出力してください。ただし、depth_cx_removedに1つも注目している区間の要素が含まれていない場合は高さは0としてください。
# グラフのタイトルは'Percentage of CNOT removed by AQCEL per section when divided into 16 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Percentage of CNOT removed by AQCEL per section'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np
import math

# 区間数
M = 16

# 最大値の取得
max_depth = max(max(depth_cx_removed), max(depth_cx_ori))

# 区間の境界値を計算
bins = np.linspace(0, max_depth, M+1)

# 区間に分けて要素の数をカウント
counts_depth_cx_removed, _ = np.histogram(depth_cx_removed, bins=bins)
counts_depth_cx_ori, _ = np.histogram(depth_cx_ori, bins=bins)

# パーセンテージの計算
percentage = [0 if c_cnot == 0 else c_ls/c_cnot for c_ls, c_cnot in zip(counts_depth_cx_removed, counts_depth_cx_ori)]

# x軸とy軸の値
x = [f'{math.floor(bins[i])}-{math.ceil(bins[i+1])}' for i in range(M)]
y = percentage

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CNOTs removed by AQCEL per section when divided into 16 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Percentage of CNOTs removed by AQCEL per section')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')
plt.yticks(np.arange(0, np.nanmax(y)+ 0.25, 0.25))

# グラフを保存
plt.savefig('figures/percentage_cnot_removed_per_section.png', bbox_inches='tight')

# グラフを表示
plt.show()

In [ ]:
########################################################################################
##### Percentage of CNOT removed by AQCEL per section when divided into 4 sections #####
########################################################################################

# これより上のメッセージは無視してください
# 次のようなグラフを書いてください
# Pythonでmatplotlibを用いてください。ax.は使わずplt.を用いてください
# リスト'depth_cx_removed', 'depth_cx_ori'の中身はそれぞれCNOTの取り除かれた位置、CNOTの位置(depth）を表しています。リストは自分で用意するのでコードに含めないでください
# depthが0から最大値までの間をM個の区間に区切ってください。ただし、区間の端点は整数になるように区切ってください。また、最初の区間の始点は0になるようにしてください。また、最後の区間では最後の値も含むようにしてください。ここではM=4とします
# depth_cx_removedリストのうち、注目している区間の範囲内にある要素の数をカウントし合計してください。m個あったとします。同じ区間について、今度はdepth_cx_oriに何個あるかカウントし合計してください。l個とします。あれば横軸（depth)がnに対して高さがm/lとなるような棒グラフを出力してください。ただし、depth_cx_removedに1つも注目している区間の要素が含まれていない場合は高さは0としてください。
# グラフのタイトルは'Percentage of CNOT removed by AQCEL per section when divided into 4 sections'
# グラフのx軸は'Segment of depth'
# グラフのy軸は'Percentage of CNOT removed by AQCEL per section'
# x軸の目盛りは区間の始まりと終わりの値がわかるように表示してください。例えば"0-20"のように表示してください。隣り合う文字がかぶるようなら文字サイズか文字の向きで調整してください。
# y軸の目盛りは0から始めて0.1の倍数ごとに表示してください
# 適切な名前をつけて保存してください。ただし他にも似たようなグラフをたくさん作るので被らないようになるべく細かく指定して名前をつけてください。保存場所は'figures/'です
# グラフの色は緑にしてください
# debugしやすいように、代入した変数全てについてできるだけ細かくprint関数で出力してください。また、printされた値が何かわかるように簡単な説明も一緒にprintしてください。

import matplotlib.pyplot as plt
import numpy as np
import math


# 区間数
M = 4

# 最大値の取得
max_depth = max(max(depth_cx_removed), max(depth_cx_ori))

# 区間の境界値を計算
bins = np.linspace(0, max_depth, M+1)

# 区間に分けて要素の数をカウント
counts_depth_cx_removed, _ = np.histogram(depth_cx_removed, bins=bins)
counts_depth_cx_ori, _ = np.histogram(depth_cx_ori, bins=bins)

# パーセンテージの計算
percentage = [0 if c_cnot == 0 else c_ls/c_cnot for c_ls, c_cnot in zip(counts_depth_cx_removed, counts_depth_cx_ori)]

# x軸とy軸の値
x = [f'{math.floor(bins[i])}-{math.ceil(bins[i+1])}' for i in range(M)]
y = percentage

# グラフを描画
plt.figure(figsize=(10, 6))
plt.bar(x, y, color='green')

# タイトルと軸のラベルを設定
plt.title('Percentage of CNOTs removed by AQCEL per section when divided into 4 sections')
plt.xlabel('Segment of depth')
plt.ylabel('Percentage of CNOTs removed by AQCEL per section')

# x軸とy軸の範囲と目盛りを設定
plt.xticks(rotation='vertical')
plt.yticks(np.arange(0, np.nanmax(y)+0.25, 0.25))

# グラフを保存
plt.savefig('figures/percentage_cnot_removed_per_section.png', bbox_inches='tight')

# グラフを表示
plt.show()